In [1]:
import spacy
import jieba

jieba.set_dictionary('data/dict.txt.big')

In [2]:
with open("data/clean/zh_book1.txt", 'r', encoding='utf-8') as file:
    text = file.read()
tokens = jieba.lcut(text, cut_all = False)

Building prefix dict from c:\Users\hugo_\Desktop\iql_lab3\data\dict.txt.big ...
Loading model from cache C:\Users\hugo_\AppData\Local\Temp\jieba.uceaac3f4f1db143f008c31874e8bc8f8.cache
Loading model cost 1.519 seconds.
Prefix dict has been built successfully.


In [25]:
punc = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.　 ﹔\n[]"
clean_tokens = [token for token in tokens if token not in punc]
text_without_punctuation = ','.join(clean_tokens)
print(len(clean_tokens),len(tokens),-len(clean_tokens)+len(tokens))

93188 132398 39210


In [26]:
for token in clean_tokens:
    print(token)

天豹
圖
清
佚名
序
若夫
指帝
天而喻
美
賦
雲雨
以
傳奇
此固
小說家
鏗金戛玉
多
存嬿婉之詞
是
世人
之
不可
與
莊語
也
然謂
柳絮
之才
罕柏舟
之操
如雲
之媛罔崩城
之烈
辭華
之
媖
多
同車
之行
苧羅之姝靡
坐檯
之守
竊香
之姬
無
墜樓
之志
琴心
之女
乏
投梭
之貞
何哉
竹箭
不花
芙
蕖
寡節
豈非
騷人墨客
借
古人
以
澆
胸中壘塊
也
細閱
此書
寓旨
隱躍
如諷
如嘲
全在
浩然之氣
耳
觀施碧霞
賣身
葬母
陷入
虎穴
終保
完璞
李榮春
仗義疏財
臨
大節
而
不可
奪
花錦章專
國柄
殘害
忠良
花子
能
倚
父勢
導情
恣
欲
強佔
人家
三十一
女
難逃
冶女
淫風
不保
妻子
父子
聚
忝
不知
儆
尚欲
弒君
僭
位
若非
萬花
老祖
預知
陶天豹
指點
諸英
雄何
知
救駕
奸相
滿門
焉能
伏誅
則施
必顯
之輩
聚集
蟠蛇山
豈
不
幾幾終
為
草寇
也
耶
予觀
古今
書籍
多無如江
如海
之
才
儒墨
旅人
集
傾國傾城
之句
未若此
天
豹圖
一書
包羅
忠孝
罔
乖
大雅
其膽
豪神
雋
可及
也
其
浩然之氣
不可
及
也
是
為序
慶
閼逢
閹茂暢
月
三影
張氏題
於
鷺門
城東
醉
墨軒
書屋
第一回
賽
專珠
施仁
濟困
淨街
王伏
霸凌貧
詩曰
雨斷
雲歸甫
作睛
夕陽
鼓角
動
高城
客愁
正得
酒排
去
草色
直疑
煙
染成
鶯
為
風
和
初命友
鷗
緣
水長
欲
尋盟
不須
苦
問春
深淺
陌上
吹蕭
已賣常
話
說
大明
成化
年間
揚州
府
江都縣
有
一
官家
子弟
姓李
名芳
字榮春
因
他
為
人
慷慨
仗義疏財
濟困扶危
憐孤惜
寡
就是
遠方
之
人流
落到
此
不能
歸
家
的
就
來
李府
向
他
求借
榮春
無不
相助
其盤資
送
他
歸家
故人
人贈
他
一個
美號
叫做
小
孟嘗君
又
一
別號
為賽
專珠
揚州
一府
無
一個
不知
其名
無人
不
感其恩
況
他
祖公
三代
俱為
司寇
司農
父
叔
二人
亦
受
司徒
司馬
之職
俱
皆
作古
家中
只有
夫人
文氏
在
堂
李榮春
娶妻
淡氏
完婚
三年
尚未
有子
榮春
在家
勤苦
讀書
今
已
中
了解
元
因
老夫
人
在
堂
不忍
遠離
膝下
所以
未曾
進京
赴試
又
且
家資